In [1]:
!pip install selenium
!pip install webdriver-manager

In [2]:
from time import sleep

from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [3]:
#Create dictionary for posts and comments
posts = dict()
comments = dict()

In [4]:
def initDriver():
    WINDOW_SIZE = "1920, 1080"
    chrome_options = Options()
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument("--disable-blink-features=AutomationControllered")
    chrome_options.add_experimental_option('useAutomationExtension', False)
    prefs = {"profile.default_content_setting_values.notifications": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_argument('disable-infobars')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [5]:
def convertToCookie(cookie):
    try:
        new_cookie = ["c_user=", "xs="]
        cookie_arr = cookie.split(";")
        for i in cookie_arr:
            if i.__contains__('c_user='):
                new_cookie[0] = new_cookie[0] + (i.strip() + ";").split("c_user=")[1]
            if i.__contains__('xs='):
                new_cookie[1] = new_cookie[1] + (i.strip() + ";").split("xs=")[1]
                if (len(new_cookie[1].split("|"))):
                    new_cookie[1] = new_cookie[1].split("|")[0]
                if (";" not in new_cookie[1]):
                    new_cookie[1] = new_cookie[1] + ";"

        conv = new_cookie[0] + " " + new_cookie[1]
        if (conv.split(" ")[0] == "c_user="):
            return
        else:
            return conv
    except:
        print("Error Convert Cookie")

In [6]:
def loginFacebookByCookie(driver ,cookie):
    try:
        cookie = convertToCookie(cookie)
        print(cookie)
        if (cookie != None):
            script = 'javascript:void(function(){ function setCookie(t) { var list = t.split("; "); console.log(list); for (var i = list.length - 1; i >= 0; i--) { var cname = list[i].split("=")[0]; var cvalue = list[i].split("=")[1]; var d = new Date(); d.setTime(d.getTime() + (7*24*60*60*1000)); var expires = ";domain=.facebook.com;expires="+ d.toUTCString(); document.cookie = cname + "=" + cvalue + "; " + expires; } } function hex2a(hex) { var str = ""; for (var i = 0; i < hex.length; i += 2) { var v = parseInt(hex.substr(i, 2), 16); if (v) str += String.fromCharCode(v); } return str; } setCookie("' + cookie + '"); location.href = "https://mbasic.facebook.com"; })();'
            driver.execute_script(script)
            sleep(5)
    except:
        print("Error login")

In [7]:
def checkLiveClone(driver):
    try:
        driver.get("https://mbasic.facebook.com/")
        sleep(3)
        driver.get("https://mbasic.facebook.com/")
        sleep(3)
        elementLive = driver.find_elements(by=By.XPATH, value='//a[contains(@href, "/messages/")]')
        if (len(elementLive) > 0):
            print("Live")
            return True
        return False
    except:
        print("Error Check Live")

In [8]:
def loginFacebookByCookie(driver ,cookie):
    try:
        cookie = convertToCookie(cookie)
        if (cookie != None):
            script = 'javascript:void(function(){ function setCookie(t) { var list = t.split("; "); console.log(list); for (var i = list.length - 1; i >= 0; i--) { var cname = list[i].split("=")[0]; var cvalue = list[i].split("=")[1]; var d = new Date(); d.setTime(d.getTime() + (7*24*60*60*1000)); var expires = ";domain=.facebook.com;expires="+ d.toUTCString(); document.cookie = cname + "=" + cvalue + "; " + expires; } } function hex2a(hex) { var str = ""; for (var i = 0; i < hex.length; i += 2) { var v = parseInt(hex.substr(i, 2), 16); if (v) str += String.fromCharCode(v); } return str; } setCookie("' + cookie + '"); location.href = "https://mbasic.facebook.com"; })();'
            driver.execute_script(script)
            sleep(5)
    except:
        print("Error login")

In [9]:
def checkLiveCookie(driver, cookie):
    try:
        driver.get('https://mbasic.facebook.com/')
        sleep(3)
        driver.get('https://mbasic.facebook.com/')
        sleep(3)
        loginFacebookByCookie(driver ,cookie)

        return checkLiveClone(driver)
    except:
        print("Error Check Live")

In [10]:
def getCommentByPost(driver, postId, target = 500):
    try:
        driver.get(posts[postId])
        try:
            commentTags = driver.find_elements(by=By.XPATH, value='//div[contains(@data-sigil, "comment-body")]')
            if (len(commentTags) < target):
                nextButton = driver.find_elements(by=By.XPATH, value='//*[contains(@id,"see_next")]/a')
                if(len(nextButton)==1):
                    nextButton[0].click()
                    sleep(3)
                repliesButtons = driver.find_elements(by=By.XPATH, value='//a[contains(@href,"comment/replies/?ctoken=")]')
                while (len(repliesButtons) > 0):
                    for repliesButton in repliesButtons:
                        repliesButton.click()
                        sleep(3)
                    repliesButtons = driver.find_elements(by=By.XPATH, value='//a[contains(@href,"comment/replies/?ctoken=")]')
        except:
            print("Error")
        commentTags = driver.find_elements(by=By.XPATH, value='//div[contains(@data-sigil, "comment-body")]')
        if (len(commentTags)):
            for comment in commentTags:
                commentId = comment.get_attribute('data-commentid')
                if(commentId and len(comments) < target):
                    commentContent = comment.text
                    innerLinks = comment.find_elements(by=By.TAG_NAME, value='a')
                    if(len(innerLinks)):
                        for innerLink in innerLinks:
                            commentContent = commentContent.replace(innerLink.text, '')
                    comments[commentId] = commentContent
    except:
        print("Error")

In [11]:
def getPostsByFanpage(driver, pageId, amount):
    driver.get("https://touch.facebook.com/" + pageId)
    while len(posts) < amount:
        sleep(3)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        shareButtons = driver.find_elements(by=By.XPATH, value='//a[contains(@href, "/sharer.php")]')
        if (len(shareButtons)):
            for shareButton  in shareButtons:
                postId = shareButton.get_attribute('href').split('sid=')[1].split('&')[0]
                commentButtons = driver.find_elements(by=By.XPATH, value='//a[contains(@data-click, "click_comment_ufi") and contains(@data-click, "'+str(postId)+'")]')
                if (len(commentButtons) == 1 and len(posts) < amount):
                    posts[postId] = commentButtons[0].get_attribute('href')

In [12]:
cookie = "c_user=100082982745987; xs=47%3A3rDjZrheBR82cA%3A2%3A1657434894%3A-1%3A-1;"
driver = initDriver()
isLive = checkLiveCookie(driver, cookie)
if (isLive):
    getPostsByFanpage(driver, 'Theanh28/photos/a.1509486855763896/6306054136107120', 1)
    for postId in posts:
        getCommentByPost(driver, postId, 500)

[WDM] - ====== WebDriver manager ======


2022-07-10 16:17:04,936 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-10 16:17:04,939 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-10 16:17:04,941 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - Driver [C:\Users\Dang Khoa\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


2022-07-10 16:17:05,149 INFO Driver [C:\Users\Dang Khoa\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
Live


In [13]:
len(posts)

1

In [14]:
len(comments)

113

In [15]:
data = comments.values()
len(data)

113

In [16]:
import pandas as pd   
df = pd.DataFrame(data, columns=['comment'])
df.to_csv('comments-predict.csv')
df = pd.read_csv('comments-predict.csv')
# Delete index column
df.drop(df.columns[[0]], axis=1, inplace=True)
df

2022-07-10 16:19:20,905 INFO NumExpr defaulting to 8 threads.


,comment
0,NaN
1,"Công nhận, mấy bạn nữ ở nhà mặc đồ ngủ nhìn dễ..."
2,"đệm vào chả vậy , ngực như cái chén lót thêm ..."
3,"khi chưa diện đồ thì nhếch nhác, phèn ịa .. nh..."
4,"Hôm nay All In T1, SGB kiếm tí vốn gập theo U1..."
...,...
108,1 chút 1 chút thôi:))))
109,Hi
110,haha ơi chời à hog nhận ra thiệt lun á
111,chúa hè rứ thôi


In [17]:
!pip install emoji
!pip install underthesea

In [18]:
import pandas as pd
import regex as re
import emoji
import string
from underthesea import word_tokenize
from underthesea import sentiment

In [19]:
# Process data crawl before model

# Basic preprocessing as lower string
def pre_processing(text):
    return text.str.lower()

df['comment'] = pre_processing(df['comment'])

# Delete NaN and empty row
df = df[df.comment != '']
df = df.dropna(subset=['comment'])
df = df[df['comment'].apply(lambda comment: len(str(comment)) >= 5)]

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub("", text)

df['comment'] = df['comment'].map(lambda comment: remove_emoji(comment))

df['comment'] = df['comment'].map(lambda comment: comment.replace(string.punctuation,''))
df['comment'] = df['comment'].map(lambda comment: comment.replace('_',' '))

# Word segmentation
df['segment'] = df['comment'].map(lambda comment: word_tokenize(comment))

# Remove vietnamese stopwords - https://github.com/stopwords/vietnamese-stopwords

# Read vietnamese stopwords file
stopwords_df = pd.read_table('vietnamese-stopwords.txt')
stopwords = stopwords_df.values.flatten()

# Remove vietnamese stopwords function
def remove_stopwords(segments):
    return [item for item in segments if item not in stopwords]

df['segment'] = df['segment'].map(lambda segments: remove_stopwords(segments))

def process(text):
    text = re.sub('&lt;/?.*?&gt;', ' &lt;&gt; ', str(text))
    text = re.sub('(\\d|\\W)+', ' ', str(text))
    return text

df['segment'] = df['segment'].apply(lambda x: process(x))
df

C:\Users\Dang Khoa\AppData\Local\Temp\ipykernel_5456\1010956789.py:15: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub("", text)


,comment,segment
1,"công nhận, mấy bạn nữ ở nhà mặc đồ ngủ nhìn dễ...",công nhận mấy nữ mặc đồ ngủ dễ thương kiểu te...
2,"đệm vào chả vậy , ngực như cái chén lót thêm ...",đệm chả ngực chén lót vải vd bát úp
3,"khi chưa diện đồ thì nhếch nhác, phèn ịa .. nh...",diện đồ nhếch nhác phèn ịa diện đồ phèn ịa y ...
4,"hôm nay all in t1, sgb kiếm tí vốn gập theo u1...",hôm nay all in t sgb kiếm tí vốn gập u việt n...
5,diện áo đẹp chất lượng nè <3 #1xxk <3,diện áo đẹp chất lượng nè xxk
...,...,...
106,em á,
107,sương sương thôi?,sương sương
108,1 chút 1 chút thôi:)))),chút chút
110,haha ơi chời à hog nhận ra thiệt lun á,haha chời hog thiệt lun


In [20]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
import numpy as np
import nltk
import re
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import LinearSVC
import joblib

In [21]:
data = pd.read_csv("comments.csv")
data

,comment,segment,emotion
0,hôm nay đi châu đốc về ghé qua quán . hủ tiếu ...,"['hôm nay', 'đi', 'châu đốc', 'ghé', 'quán', '...",neg
1,đây hẳn hoi là sưởng bánh bao nên mua gần như ...,"['hẳn hoi', 'sưởng', 'bánh bao', 'mua', 'giá b...",pos
2,không gian sang choảnh mà giá k hề sang tí nào...,"['không gian', 'choảnh', 'giá', 'k', 'hề', 'tí...",pos
3,nhưng người ta xinh hơn c :)))),"['người ta', 'xinh', 'c', ':))))']",neu
4,hãy xem xét kỹ lại bản thân đi mấy thằng đb cơ...,"['xem xét', 'kỹ', 'đi', 'mấy', 'thằng', 'đb']",neu
...,...,...,...
109824,ý nghĩ tiêu cực.,"['ý nghĩ', 'tiêu cực', '.']",neu
109825,chết cụ lũ súc sinh chúng m,"['chết', 'cụ', 'lũ', 'súc sinh', 'm']",neg
109826,"thôi ông anh ơi, người bình thường chứ có phả...","[',', 'bình thường', 'thần thánh', 'đ', 'đòi',...",neu
109827,tuổi mới phải thật mạnh mẽ anh nhé . hãy nhớ r...,"['mạnh mẽ', '.', 'chúng em', 'đằng', 'sauuu', ...",pos


In [22]:
data['segment'] = data['segment'].apply(lambda x: process(x))
data

,comment,segment,emotion
0,hôm nay đi châu đốc về ghé qua quán . hủ tiếu ...,hôm nay đi châu đốc ghé quán hủ tiếu bình thư...,neg
1,đây hẳn hoi là sưởng bánh bao nên mua gần như ...,hẳn hoi sưởng bánh bao mua giá buôn bánh hàng...,pos
2,không gian sang choảnh mà giá k hề sang tí nào...,không gian choảnh giá k hề tí hợp vs mấy hẹn ...,pos
3,nhưng người ta xinh hơn c :)))),người ta xinh c,neu
4,hãy xem xét kỹ lại bản thân đi mấy thằng đb cơ...,xem xét kỹ đi mấy thằng đb,neu
...,...,...,...
109824,ý nghĩ tiêu cực.,ý nghĩ tiêu cực,neu
109825,chết cụ lũ súc sinh chúng m,chết cụ lũ súc sinh m,neg
109826,"thôi ông anh ơi, người bình thường chứ có phả...",bình thường thần thánh đ đòi học câu thi học ...,neu
109827,tuổi mới phải thật mạnh mẽ anh nhé . hãy nhớ r...,mạnh mẽ chúng em đằng sauuu i love sơn tùng m...,pos


In [23]:
segment = data['segment']
emotion = data['emotion']

vectorizer = TfidfVectorizer(
    binary=True, ngram_range=(1, 2),
    sublinear_tf=True,
    use_idf=True)

X = vectorizer.fit_transform(segment)

In [24]:
# Save vectorizer
vectorizer_file = 'vectorizer_file.sav'
joblib.dump(vectorizer, vectorizer_file)

# Load vectorizer
loaded_vectorizer_file = joblib.load('vectorizer_file.sav')

In [25]:
#data test and data train 
X_train, X_test, y_train, y_test = train_test_split(
    X, emotion, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(87863, 700968) (87863,)
(21966, 700968) (21966,)


In [26]:
#model LinearSVC
linear = LinearSVC(C=1, max_iter=1000)
linear = linear.fit(X_train, y_train)

In [27]:
linearModel = 'linear_model.sav'
joblib.dump(linear, linearModel)

['linear_model.sav']

In [28]:
loaded_linearModel = joblib.load('linear_model.sav')

In [29]:
#Test
y_pred = loaded_linearModel.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy score :",accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.75      0.70      0.73      8084
         neu       0.70      0.82      0.75      7381
         pos       0.80      0.71      0.75      6501

    accuracy                           0.74     21966
   macro avg       0.75      0.74      0.74     21966
weighted avg       0.75      0.74      0.74     21966

Accuracy score : 0.7431485022307202


In [30]:
output = loaded_linearModel.predict(loaded_vectorizer_file.transform(df['segment']))
print(output)

['pos' 'neu' 'neg' 'neu' 'pos' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg' 'neu'
 'neg' 'pos' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg' 'neg' 'neg' 'neg' 'neu'
 'neu' 'pos' 'neu' 'neu' 'pos' 'neg' 'neg' 'pos' 'neu' 'neg' 'neu' 'neu'
 'neg' 'neu' 'neg' 'neu' 'neg' 'neu' 'neu' 'neg' 'neu' 'neu' 'pos' 'neg'
 'neu' 'neu' 'pos' 'neu' 'neg' 'neu' 'neu' 'pos' 'neu' 'neu' 'neu' 'neg'
 'neu' 'neg' 'neu' 'neg' 'neu' 'neg' 'neu' 'neu' 'neu' 'neu' 'neu' 'neu'
 'neu' 'neu' 'pos' 'neg' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg' 'neu' 'neu'
 'neg']


In [31]:
df.insert(2, "emotion", output)
# df.drop("emotion", axis=1, inplace=True)
df

,comment,segment,emotion
1,"công nhận, mấy bạn nữ ở nhà mặc đồ ngủ nhìn dễ...",công nhận mấy nữ mặc đồ ngủ dễ thương kiểu te...,pos
2,"đệm vào chả vậy , ngực như cái chén lót thêm ...",đệm chả ngực chén lót vải vd bát úp,neu
3,"khi chưa diện đồ thì nhếch nhác, phèn ịa .. nh...",diện đồ nhếch nhác phèn ịa diện đồ phèn ịa y ...,neg
4,"hôm nay all in t1, sgb kiếm tí vốn gập theo u1...",hôm nay all in t sgb kiếm tí vốn gập u việt n...,neu
5,diện áo đẹp chất lượng nè <3 #1xxk <3,diện áo đẹp chất lượng nè xxk,pos
...,...,...,...
106,em á,,neu
107,sương sương thôi?,sương sương,neg
108,1 chút 1 chút thôi:)))),chút chút,neu
110,haha ơi chời à hog nhận ra thiệt lun á,haha chời hog thiệt lun,neu


In [32]:
#LogisticRegression model
Logistic = LogisticRegression(max_iter=1000)
Logistic.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [33]:
logisticModel = 'logistic_model.sav'
joblib.dump(Logistic, logisticModel)

['logistic_model.sav']

In [34]:
loaded_logisticModel = joblib.load('logistic_model.sav')

In [35]:
y_pred = loaded_logisticModel.predict(X_test)
print("Accuracy LogisticRegression:", accuracy_score(y_test,y_pred ))
print(classification_report(y_test, y_pred))

Accuracy LogisticRegression: 0.7473367932258945
              precision    recall  f1-score   support

         neg       0.76      0.70      0.73      8084
         neu       0.69      0.85      0.76      7381
         pos       0.82      0.69      0.75      6501

    accuracy                           0.75     21966
   macro avg       0.76      0.75      0.75     21966
weighted avg       0.76      0.75      0.75     21966



In [36]:
outputLogistic = loaded_logisticModel.predict(loaded_vectorizer_file.transform(df['segment']))
print(outputLogistic)

['pos' 'neu' 'neg' 'neu' 'pos' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg' 'neu'
 'neg' 'pos' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg' 'neg' 'neg' 'neg' 'neg'
 'neu' 'pos' 'neu' 'neu' 'pos' 'neg' 'neg' 'pos' 'neu' 'neg' 'neu' 'neu'
 'neg' 'neu' 'neg' 'neu' 'neg' 'neu' 'neu' 'neg' 'neu' 'neu' 'pos' 'neg'
 'neu' 'neu' 'pos' 'neu' 'neg' 'neu' 'neu' 'pos' 'neu' 'neu' 'neu' 'neg'
 'neu' 'neu' 'neu' 'neg' 'neu' 'neg' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg'
 'neu' 'neu' 'neg' 'neu' 'neu' 'neu' 'neu' 'neu' 'neu' 'neg' 'neu' 'pos'
 'neu']


In [37]:
df.insert(3, "emotionLogistic", output)
# df.drop("emotion", axis=1, inplace=True)
df

,comment,segment,emotion,emotionLogistic
1,"công nhận, mấy bạn nữ ở nhà mặc đồ ngủ nhìn dễ...",công nhận mấy nữ mặc đồ ngủ dễ thương kiểu te...,pos,pos
2,"đệm vào chả vậy , ngực như cái chén lót thêm ...",đệm chả ngực chén lót vải vd bát úp,neu,neu
3,"khi chưa diện đồ thì nhếch nhác, phèn ịa .. nh...",diện đồ nhếch nhác phèn ịa diện đồ phèn ịa y ...,neg,neg
4,"hôm nay all in t1, sgb kiếm tí vốn gập theo u1...",hôm nay all in t sgb kiếm tí vốn gập u việt n...,neu,neu
5,diện áo đẹp chất lượng nè <3 #1xxk <3,diện áo đẹp chất lượng nè xxk,pos,pos
...,...,...,...,...
106,em á,,neu,neu
107,sương sương thôi?,sương sương,neg,neg
108,1 chút 1 chút thôi:)))),chút chút,neu,neu
110,haha ơi chời à hog nhận ra thiệt lun á,haha chời hog thiệt lun,neu,neu
